## Coursera Captstone Clustering Course 3 excercises

### click <a href="#task1">here</a> to go to Task 1
### click <a href="#task2">here</a> to go to Task 2
### click <a href="#task3">here</a> to go to Task 3

<a id="task1"></a>

###  Task 1 - web scraping

In [41]:
import requests, csv

In [4]:
import bs4 # beautifulsoup 4 package
from bs4 import BeautifulSoup as bsoup

In [316]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [366]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [28]:
r = requests.get(url)

In [29]:
html = r.text

In [30]:
soup = bsoup(html,'html.parser')

In [47]:
post_codes_table = soup.find_all("table")[0]

In [90]:
temp_file = "/tmp/zipcodes.csv"
with open(temp_file,"wt") as f:
    w = csv.writer(f)
    
    for row in post_codes_table.select("tbody > tr"):
        row_text = row.get_text(",,,",strip=True).split(",,,")
        w.writerow(row_text)
        # print(row_text)

In [91]:
df = pd.read_csv(temp_file)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [92]:
df.shape

(288, 3)

In [190]:
df_valid_bor = df[df['Borough'] != "Not assigned"].reset_index(drop=True)

In [191]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [193]:
grp_post = df_valid_bor.groupby(by='Postcode')

In [194]:
trans = grp_post['Neighbourhood'].transform(lambda x: ",".join(x))

In [195]:
trans.shape

(211,)

In [196]:
df_valid_bor.shape

(211, 3)

In [197]:
df_valid_bor['Neigh2'] = trans
df_valid_bor.head()

,Postcode,Borough,Neighbourhood,Neigh2
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,Regent Park,"Harbourfront,Regent Park"
4,M6A,North York,Lawrence Heights,"Lawrence Heights,Lawrence Manor"


In [198]:
df_valid_bor.drop(columns=["Neighbourhood"], inplace=True)

In [199]:
df_valid_bor.head()

,Postcode,Borough,Neigh2
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [200]:
df_valid_bor.rename(columns={'Neigh2': 'Neighbourhood'}, inplace=True)

In [201]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [202]:
df_valid_bor.drop_duplicates(inplace=True)

In [203]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Not assigned


In [204]:
df_valid_bor.reset_index(drop=True, inplace=True)

In [205]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [206]:
df_valid_bor.shape

(103, 3)

In [207]:
df_valid_bor['Neighbourhood'] = df_valid_bor.apply(lambda row: row['Borough'] if row['Neighbourhood'] == "Not assigned" else row['Neighbourhood'], axis=1)

In [208]:
df_valid_bor.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [209]:
df_valid_bor.shape

(103, 3)

<a id="task2"></a>

### Task 2 - get latitude and longitude

In [180]:
import geocoder

In [211]:
df2 = df_valid_bor.copy(deep=True)

In [212]:
lats = []
longs = []
for postal_code in df2['Postcode']:

    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    neigh_lat = lat_lng_coords[0]
    neigh_long = lat_lng_coords[1]
    print(lat_lng_coords)
    # df2 = df2.append({'Latitude': neigh_lat, 'Longitude': neigh_long}, ignore_index=False)
    lats.append(neigh_lat)
    longs.append(neigh_long)

[43.752440000000036, -79.32927072599995]
[43.730420577000075, -79.31331999999998]
[43.65512000000007, -79.36263979699999]
[43.72312500000004, -79.45158914699994]
[43.66110229800006, -79.39103499999999]
[43.662242201000026, -79.52837877199994]
[43.811525000000074, -79.19551721399995]
[43.74919500000004, -79.36190541699995]
[43.707535000000064, -79.31177329699995]
[43.65736301100003, -79.37817999999999]
[43.70799000000005, -79.44836733199998]
[43.64969222700006, -79.55394499999994]
[43.78573000000006, -79.15874999999994]
[43.72142500000007, -79.34345278999996]
[43.68966500000005, -79.30716910999996]
[43.65121000000005, -79.37548057699996]
[43.692105179000066, -79.43035499999996]
[43.648573449000025, -79.57824999999997]
[43.76569000000006, -79.17525603599995]
[43.67684518300007, -79.29522499999996]
[43.64516015600003, -79.37367499999993]
[43.68864000000008, -79.45101525299998]
[43.76835912100006, -79.21758999999997]
[43.70976500000006, -79.36390090899994]
[43.65609081300005, -79.384929999

In [213]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [214]:
df2['Latitude'] = lats

In [215]:
df2['Longitude'] = longs

In [216]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [217]:
df2.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653690,-79.511117
99,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.632765,-79.489601
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.624630,-79.526950


In [219]:
df2.shape

(103, 5)

In [233]:
df2.dtypes

Postcode          object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

<a id="task3"></a>

### Task 3 - Clustering

In [236]:
from geopy.geocoders import Nominatim

In [238]:
import folium

In [329]:
df3 = df2.copy(deep=True)
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035


In [330]:
df3.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)

In [331]:
df3.tail()

,Postcode,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653690,-79.511117
99,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.632765,-79.489601
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.624630,-79.526950


In [332]:
# get Toronto's lat and long to draw a map
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [333]:
# choosing only boroughs with the name Toronto in it
df4 = df3[df3['Borough'].str.match('.*Toronto.*')]

In [334]:
df4.shape

(38, 5)

In [335]:
df4.reset_index(drop=True, inplace=True)

In [336]:
df4.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180
2,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
3,M4E,East Toronto,The Beaches,43.676845,-79.295225
4,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675


In [337]:
df4.tail()

,Postcode,Borough,Neighborhood,Latitude,Longitude
33,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.648690,-79.385440
34,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
35,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648280,-79.381461
36,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440


In [339]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7, 
        parse_html=False).add_to(map_toronto)
map_toronto

In [340]:
# explore a single neighborhood
df4.loc[0,'Neighborhood']

'Harbourfront,Regent Park'

In [341]:
neigh_lat = df4.loc[0,'Latitude']
neigh_lng = df4.loc[0,'Longitude']
print(neigh_lat, neigh_lng)

43.65512000000007 -79.36263979699999


In [342]:
# Foursquare credentials
CLIENT_ID = 'HRZNHDDCZQJ4U4NWRR1LYPJJVSYSPJCAHUW22JOZADGFOVL1' # your Foursquare ID
CLIENT_SECRET = '0THVETABYGSJYWMMTFCMLWJQNKJ4GT42DVDSIUDMYQC2YMEG' # your Foursq uare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HRZNHDDCZQJ4U4NWRR1LYPJJVSYSPJCAHUW22JOZADGFOVL1
CLIENT_SECRET:0THVETABYGSJYWMMTFCMLWJQNKJ4GT42DVDSIUDMYQC2YMEG


In [343]:
# get top 10 venues in the above neighbourhood within a 500 meter radius
RADIUS=500
LIMIT=10
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neigh_lat, neigh_lng, VERSION, RADIUS, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=HRZNHDDCZQJ4U4NWRR1LYPJJVSYSPJCAHUW22JOZADGFOVL1&client_secret=0THVETABYGSJYWMMTFCMLWJQNKJ4GT42DVDSIUDMYQC2YMEG&ll=43.65512000000007,-79.36263979699999&v=20180605&radius=500&limit=10


In [344]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d3e8973018cbb002c42e09c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 192,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [345]:
# function to extract category of the venue
def get_category_type(row): 
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [346]:
from pandas.io.json import json_normalize

In [347]:
# explore the results above from foursquare
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Cocina Economica,Mexican Restaurant,43.654959,-79.365657
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [348]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [349]:
# now we explore all neighbourhoods
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name, lat, lng)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                VERSION,
                lat,
                lng,
                radius,
                LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                      'Neighborhood Latitude',
                      'Neighborhood Longitude',
                      'Venue',
                      'Venue Latitude',
                      'Venue Longitude',
                      'Venue Category']
    
    return(nearby_venues)

In [296]:
toronto_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                )

Harbourfront,Regent Park 43.65512000000007 -79.36263979699999
Ryerson,Garden District 43.65736301100003 -79.37817999999999
St. James Town 43.65121000000005 -79.37548057699996
The Beaches 43.67684518300007 -79.29522499999996
Berczy Park 43.64516015600003 -79.37367499999993
Central Bay Street 43.65609081300005 -79.38492999999994
Christie 43.66878132800008 -79.42070999999999
Adelaide,King,Richmond 43.649515000000065 -79.38250344699998
Dovercourt Village,Dufferin 43.66508694300006 -79.43870499999997
Harbourfront East,Toronto Islands,Union Station 43.623470000000054 -79.39150736399995
Little Portugal,Trinity 43.64848500000005 -79.41774150899994
The Danforth West,Riverdale 43.68326150000007 -79.35511999999994
Design Exchange,Toronto Dominion Centre 43.64710000000008 -79.38153109899997
Brockton,Exhibition Place,Parkdale Village 43.639410000000055 -79.42436201999999
The Beaches West,India Bazaar 43.66796500000004 -79.31467251099997
Commerce Court,Victoria Hotel 43.648205000000075 -79.378793398

In [350]:
toronto_venues.shape

(1749, 7)

In [351]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Harbourfront,Regent Park",43.65512,-79.36264,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
4,"Harbourfront,Regent Park",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [352]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,63,63,63,63,63,63
"Brockton,Exhibition Place,Parkdale Village",69,69,69,69,69,69
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",72,72,72,72,72,72
"Cabbagetown,St. James Town",41,41,41,41,41,41
Central Bay Street,100,100,100,100,100,100
"Chinatown,Grange Park,Kensington Market",98,98,98,98,98,98
Christie,10,10,10,10,10,10


In [353]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 216 uniques categories.


In [354]:
# analysis of each neighbourhood
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [355]:
toronto_onehot.shape

(1749, 216)

In [356]:
toronto_onehot.columns

Index(['Yoga Studio', 'Afghan Restaurant', 'American Restaurant',
       'Antique Shop', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store',
       'Asian Restaurant', 'Athletics & Sports', 'BBQ Joint',
       ...
       'Thrift / Vintage Store', 'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Veterinarian', 'Video Game Store',
       'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint'],
      dtype='object', length=216)

In [357]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.00,0.000000,0.030000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.015873,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.014493,0.00,0.014493,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.028986,0.000000,0.000000,0.014493,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.030000,0.000000,0.010000,0.00,0.000000,0.020000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.013889,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.013889,0.000000,...,0.000000,0.00,0.000000,0.013889,0.013889,0.013889,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.024390,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.000000,0.000000,...,0.000000,0.01,0.000000,0.000000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.010204,0.00,0.020408,0.000000,0.000000,...,0.010204,0.00,0.000000,0.000000,0.051020,0.000000,0.000000,0.040816,0.010204,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.012658,0.012658,0.000000,0.000000,0.00,0.012658,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012658,0.012658,0.000000,0.012658


In [358]:
toronto_grouped.shape

(38, 216)

In [359]:
# print top 5 in each neighborhood
num_top_venues = 5
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.07
2                Hotel  0.05
3  Japanese Restaurant  0.03
4         Burger Joint  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.10
1      Restaurant  0.05
2          Bakery  0.05
3    Cocktail Bar  0.05
4  Breakfast Spot  0.03


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1              Restaurant  0.06
2                    Café  0.06
3  Furniture / Home Store  0.06
4          Sandwich Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1          Bar  0.06
2         Café  0.05
3   Steakhouse  0.04
4        Hotel  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                  venue  freq
0           Coffee Shop  0.11
1    Italian Restaurant  0.

In [360]:
# func to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [361]:
# top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]): 
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Asian Restaurant,Restaurant,Steakhouse,Japanese Restaurant,Bar,Burger Joint,Deli / Bodega
1,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Hotel,Cheese Shop,Café,Seafood Restaurant,Lounge,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Restaurant,Café,Furniture / Home Store,Sandwich Place,Bar,Supermarket,Italian Restaurant,Beer Bar,Vegetarian / Vegan Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Bar,Café,Steakhouse,Hotel,Italian Restaurant,American Restaurant,Pub,Pizza Place,Sushi Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Restaurant,Café,Gym / Fitness Center,Bar,Pub,Park,Bakery,Sandwich Place


In [362]:
# cluster the neighborhoods
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [363]:
kmeans.labels_[0:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4,
       3, 0, 0, 4, 0, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [364]:
df4.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [365]:
# add cluster column to df
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df4
# merge toronto_grouped with toronto_data to add latitude/longitude for each nei ghborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640,0,Coffee Shop,Gym / Fitness Center,Restaurant,Breakfast Spot,Event Space,Spa,Health Food Store,Mexican Restaurant,Food Truck,Electronics Store
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Theater,Plaza,Italian Restaurant,Tea Room
2,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,0,Coffee Shop,Restaurant,Hotel,Café,Gastropub,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Breakfast Spot,Clothing Store
3,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,0,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Hotel,Cheese Shop,Café,Seafood Restaurant,Lounge,Breakfast Spot


In [368]:
# visualize clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)] 
colors_array = cm.rainbow(np.linspace(0, 1, len(ys))) 
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged ['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True, 
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

In [370]:
toronto_merged['Cluster Labels'].unique()

array([0, 3, 2, 1, 4])

In [371]:
# examine clusters
# cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Gym / Fitness Center,Restaurant,Breakfast Spot,Event Space,Spa,Health Food Store,Mexican Restaurant,Food Truck,Electronics Store
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Fast Food Restaurant,Theater,Plaza,Italian Restaurant,Tea Room
2,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Gastropub,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Breakfast Spot,Clothing Store
3,East Toronto,0,Health Food Store,Other Great Outdoors,Trail,Pub,Wings Joint,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
4,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Hotel,Cheese Shop,Café,Seafood Restaurant,Lounge,Breakfast Spot
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Middle Eastern Restaurant,Tea Room,Ice Cream Shop,Sushi Restaurant,Plaza,Bookstore
6,Downtown Toronto,0,Café,Grocery Store,Baby Store,Playground,Italian Restaurant,Coffee Shop,Athletics & Sports,Falafel Restaurant,Farm,Farmers Market
7,Downtown Toronto,0,Coffee Shop,Café,Hotel,Asian Restaurant,Restaurant,Steakhouse,Japanese Restaurant,Bar,Burger Joint,Deli / Bodega
8,West Toronto,0,Park,Smoke Shop,Pizza Place,Brazilian Restaurant,Middle Eastern Restaurant,Fast Food Restaurant,Bar,Bank,Bakery,Furniture / Home Store
9,Downtown Toronto,0,Harbor / Marina,Café,Music Venue,Wings Joint,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [372]:
# cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,1,Pharmacy,Wings Joint,Elementary School,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [373]:
# cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Home Service,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [374]:
# cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,3,Bus Line,Swim School,Wings Joint,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [375]:
# cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,West Toronto,4,Convenience Store,Park,Wings Joint,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
22,Central Toronto,4,Playground,Gym Pool,Park,Garden,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
32,Downtown Toronto,4,Bank,Playground,Park,Building,Ethiopian Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


## end of project !